In [15]:
# system imports
import os
import sys

# data science
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import seaborn as sns

# signal processing
import scipy
from scipy import signal
from scipy.ndimage import label
from scipy.stats import zscore
from scipy.interpolate import interp1d
from scipy.integrate import trapz


# misc
import warnings

import glob

##
import pytz
import datetime as dt
import math
import seaborn as sns

import pickle


#import seaborn

In [16]:
#style settings
sns.set(style='whitegrid', rc={'axes.facecolor': 'white'})

#sns.set_style({'font.family':'Arial', 'font.serif':'Times New Roman'})

In [17]:
def read_manipulate_psychopy(psychopy_path):
    
    psychopy_df = pd.read_csv(psychopy_path)
    col_list = [col for col in psychopy_df.columns if col.endswith('_ts')]
    col_list.insert(0, "Reference_time")
    psychopy_df_selected = psychopy_df[col_list]
    
    return psychopy_df_selected


def get_nonNan_list_psychopy(psychopy_df, col_name):
    
    
    selected_vals=[val for val in psychopy_df[col_name].to_list() if not(math.isnan(val))]
    
    return selected_vals

In [18]:
def get_offset_timestamp(timestamp, offset_mins):
    
    time_zone = 'Europe/Berlin'
    tz = pytz.timezone(time_zone)
    local_time = dt.datetime.fromtimestamp(timestamp, tz)
    time_change = dt.timedelta(minutes=offset_mins)
    new_time = local_time + time_change
    new_timestamp =  dt.datetime.timestamp(new_time)
    return new_timestamp


def get_time_diff_saliva_ts(timestamp_1, timestamp_2):
    
    time_zone = 'Europe/Berlin'
    tz = pytz.timezone(time_zone)
    local_time_1 = dt.datetime.fromtimestamp(timestamp_1, tz)
    local_time_2 = dt.datetime.fromtimestamp(timestamp_2, tz)
    difference = local_time_2 - local_time_1
    diff_seconds = difference.total_seconds() 
    diff_minutes =diff_seconds/60
    
    return diff_minutes

In [19]:
data_dir_path = "/media/muhammad/d5e4618d-a960-4f0d-a521-1c804bbebbb4/data_sony_digiRelax/wp3"
#data_dir_path = "D:/Datasets/data_sony_digiRelax/study"
sub_dirs=next(os.walk(data_dir_path))[1]
sub_dirs.sort(reverse=False)

dest_path = "/media/muhammad/d5e4618d-a960-4f0d-a521-1c804bbebbb4/data_sony_digiRelax/result_psychopy_extracted_info/saliva_time_mins"

In [25]:

list_saliva_probe_ts= ["saliva_probe_1_ts", "saliva_probe_2_ts","saliva_probe_3_ts", "saliva_probe_4_ts"]
extracted_info_dict = {}
all_files_list=[]
extracted_info_dict["file_name"]=[]

for sub_dir in sub_dirs:
    
    sub_dir_path= os.path.join(data_dir_path, sub_dir)
    psychopy_file_path = os.path.join(sub_dir_path, "Psychopy_data")
    psychopy_file_path = glob.glob(psychopy_file_path+ "/*.csv")[0]
    
    dir_path, file_name=os.path.split(psychopy_file_path)
    extracted_info_dict["file_name"].append(file_name)
    sub_id=file_name[:-4]
    print(sub_id)
    
    
    psychopy_df = read_manipulate_psychopy(psychopy_file_path)
    
    ind_file_list = []
    for ind, sel_saliva in enumerate(list_saliva_probe_ts):
        
    
        if ind==0:

            prev_saliva=sel_saliva
            prev_saliva_ts = get_nonNan_list_psychopy(psychopy_df, prev_saliva)[0]

        else:
            curr_saliva = sel_saliva
            curr_saliva_ts = get_nonNan_list_psychopy(psychopy_df, curr_saliva)[0]
            time_curr_minus_prev=get_time_diff_saliva_ts(prev_saliva_ts, curr_saliva_ts)
            ind_file_list.append(time_curr_minus_prev)

            print("Time between:  " + curr_saliva +  " - " + prev_saliva + " = " + str(time_curr_minus_prev) + " mins")
            prev_saliva =curr_saliva
            prev_saliva_ts = curr_saliva_ts
            
            
        
        
    all_files_list.append(ind_file_list)
            
    print("-----------------------------------------------------------------")
    
    
extracted_info_dict["saliva_probe_2_1"] = []
extracted_info_dict["saliva_probe_3_2"] = []
extracted_info_dict["saliva_probe_4_3"] = []

for sel_file in all_files_list:
    
    #print(sel_file)
    extracted_info_dict["saliva_probe_2_1"].append(sel_file[0])
    extracted_info_dict["saliva_probe_3_2"].append(sel_file[1])
    extracted_info_dict["saliva_probe_4_3"].append(sel_file[2])


    
df_export = pd.DataFrame.from_dict(extracted_info_dict)

dest_path_saliva_time = os.path.join(dest_path,"saliva_time_interval_mins.csv")

df_export.to_csv(dest_path_saliva_time)
    

VP004_081123_DigiRelax_Experiment_2023-11-08_10h28.36.317
Time between:  saliva_probe_2_ts - saliva_probe_1_ts = 20.181002133333333 mins
Time between:  saliva_probe_3_ts - saliva_probe_2_ts = 24.436710816666668 mins
Time between:  saliva_probe_4_ts - saliva_probe_3_ts = 15.819049966666666 mins
-----------------------------------------------------------------
VP005_081123_DigiRelax_Experiment_2023-11-08_14h17.35.226
Time between:  saliva_probe_2_ts - saliva_probe_1_ts = 20.065377716666667 mins
Time between:  saliva_probe_3_ts - saliva_probe_2_ts = 24.588638166666666 mins
Time between:  saliva_probe_4_ts - saliva_probe_3_ts = 13.110345966666667 mins
-----------------------------------------------------------------
VP006_091123_DigiRelax_Experiment_2023-11-09_10h17.40.415
Time between:  saliva_probe_2_ts - saliva_probe_1_ts = 21.783582466666665 mins
Time between:  saliva_probe_3_ts - saliva_probe_2_ts = 25.96337511666667 mins
Time between:  saliva_probe_4_ts - saliva_probe_3_ts = 11.94478

IndexError: list index out of range